<a href="https://colab.research.google.com/github/VictorHolanda21/MachineLearning-PrevisaoFutebol/blob/main/7_Preparacao_Dados_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importando bibliotecas

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
url_loc_files = '/content/drive/MyDrive/Colab Notebooks/Previsao No Futebol/'

dados_elenco_merge = pd.read_csv(url_loc_files + 'dados_elencos_merge.csv')
dados_elenco_merge.head()

,nome,posicao_jogador,idade,altura_jogador,peso_jogador,nacionalidade,time,ano,classificao_time
0,Manuel Almunia,G,44.0,1.91,86.0,Espanha,arsenal,2003,1
1,Mark Howard,G,35.0,1.83,76.0,Inglaterra,arsenal,2003,1
2,Jens Lehmann1,G,51.0,1.91,87.0,Alemanha,arsenal,2003,1
3,Stuart Taylor13,G,40.0,1.96,79.0,Inglaterra,arsenal,2003,1
4,Ryan Garry,D,38.0,1.88,82.0,Inglaterra,arsenal,2003,1


# 2. Feature Engineer

Novas transformações nos dados para tentar melhorar o modelo

In [23]:
dados_elenco_merge.classificao_time.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

## 2.1 Criando grupos para classificação 

In [24]:
dados_elenco_merge['classificacao_time_2'] = pd.cut(
                                                    dados_elenco_merge.classificao_time,
                                                    [1,5,17,20],
                                                    include_lowest=True,
                                                    labels=False
                                                )
dados_elenco_merge.head()

,nome,posicao_jogador,idade,altura_jogador,peso_jogador,nacionalidade,time,ano,classificao_time,classificacao_time_2
0,Manuel Almunia,G,44.0,1.91,86.0,Espanha,arsenal,2003,1,0
1,Mark Howard,G,35.0,1.83,76.0,Inglaterra,arsenal,2003,1,0
2,Jens Lehmann1,G,51.0,1.91,87.0,Alemanha,arsenal,2003,1,0
3,Stuart Taylor13,G,40.0,1.96,79.0,Inglaterra,arsenal,2003,1,0
4,Ryan Garry,D,38.0,1.88,82.0,Inglaterra,arsenal,2003,1,0


## 2.2 Validando se a nova variável de classificação está correta


In [25]:
dados_elenco_merge.groupby(['classificao_time', 'classificacao_time_2']).ano.count()

classificao_time  classificacao_time_2
1                 0                       626
2                 0                       641
3                 0                       666
4                 0                       677
5                 0                       622
6                 1                       576
7                 1                       599
8                 1                       611
9                 1                       584
10                1                       567
11                1                       614
12                1                       615
13                1                       565
14                1                       572
15                1                       612
16                1                       584
17                1                       570
18                2                       586
19                2                       566
20                2                       594
Name: ano, dtype: int64

## 2.3 Criando coluna com flag para indicar se é estrangeiro ou não

In [26]:
dados_elenco_merge.loc[:,'estrangeiro'] = dados_elenco_merge.nacionalidade.apply(
                                                      lambda x: 1 if x != 'Inglaterra' else 0
                                                  )

dados_elenco_merge.head()

,nome,posicao_jogador,idade,altura_jogador,peso_jogador,nacionalidade,time,ano,classificao_time,classificacao_time_2,estrangeiro
0,Manuel Almunia,G,44.0,1.91,86.0,Espanha,arsenal,2003,1,0,1
1,Mark Howard,G,35.0,1.83,76.0,Inglaterra,arsenal,2003,1,0,0
2,Jens Lehmann1,G,51.0,1.91,87.0,Alemanha,arsenal,2003,1,0,1
3,Stuart Taylor13,G,40.0,1.96,79.0,Inglaterra,arsenal,2003,1,0,0
4,Ryan Garry,D,38.0,1.88,82.0,Inglaterra,arsenal,2003,1,0,0


## 2.4 Calculando a média dos atributos com base na nova classificação

In [27]:
media_classificacao_time = dados_elenco_merge.groupby(
                                                  ['ano','classificacao_time_2']
                                                ).agg({
                                                  'idade':'mean',
                                                  'altura_jogador':'mean',
                                                  'peso_jogador':'mean'
                                              }).reset_index()
media_classificacao_time.head()

,ano,classificacao_time_2,idade,altura_jogador,peso_jogador
0,2003,0,40.976190,1.818743,76.018072
1,2003,1,42.373178,1.821686,77.077151
2,2003,2,42.301370,1.812329,76.712329
3,2004,0,40.067485,1.828354,77.558282
4,2004,1,41.632836,1.822275,77.241791


## 2.5 Quantidade de jogadores no elenco para as respectivas posições na classificação

In [28]:
qtd_jogadores_posicao = (dados_elenco_merge.groupby(['ano','classificacao_time_2'])
                                            .nome
                                            .count()
                                            .reset_index()
                                            .rename(columns={'nome':'qtd_jogadores'}))

qtd_jogadores_posicao.head()

,ano,classificacao_time_2,qtd_jogadores
0,2003,0,168
1,2003,1,343
2,2003,2,73
3,2004,0,166
4,2004,1,340


## 2.6 Calcular a quantidade de times por posição na classificação

In [29]:
qtd_jogadores_posicao =  ( dados_elenco_merge.groupby(['ano','classificacao_time_2'])
                                .agg({'nome':'count', 'time':'nunique'})
                                .reset_index()
                                .rename(columns = {'nome':'qtd_jogadores'}))

qtd_jogadores_posicao.head()

,ano,classificacao_time_2,qtd_jogadores,time
0,2003,0,168,5
1,2003,1,343,12
2,2003,2,73,3
3,2004,0,166,5
4,2004,1,340,12


## 2.7 Calcular a média de jogadores por time por posição na classificação

In [30]:
qtd_jogadores_posicao.loc[:,'media_jogadores'] = qtd_jogadores_posicao.qtd_jogadores / qtd_jogadores_posicao.time
qtd_jogadores_posicao.head()

,ano,classificacao_time_2,qtd_jogadores,time,media_jogadores
0,2003,0,168,5,33.600000
1,2003,1,343,12,28.583333
2,2003,2,73,3,24.333333
3,2004,0,166,5,33.200000
4,2004,1,340,12,28.333333


## 2.8 Calcular a porcentagem de estrangeiros por cada posição

In [31]:
perct_estrangeiro = (dados_elenco_merge.groupby(['ano', 'classificacao_time_2'])
                                        .estrangeiro
                                        .value_counts(normalize=True)*100)
perct_estrangeiro

ano   classificacao_time_2  estrangeiro
2003  0                     1              63.095238
                            0              36.904762
      1                     1              54.810496
                            0              45.189504
      2                     0              52.054795
                                             ...    
2021  0                     0              37.974684
      1                     1              56.250000
                            0              43.750000
      2                     1              59.756098
                            0              40.243902
Name: estrangeiro, Length: 114, dtype: float64

In [32]:
# convertendo em pandas dataframe
pct_estrangeiro = pd.DataFrame(perct_estrangeiro)
pct_estrangeiro.head(6)

estrangeiro
ano  classificacao_time_2 estrangeiro             
2003 0                    1              63.095238
                          0              36.904762
     1                    1              54.810496
                          0              45.189504
     2                    0              52.054795
                          1              47.945205

In [33]:
# renomeando o nome da coluna
pct_estrangeiro = pct_estrangeiro.rename(columns={'estrangeiro':'pct_estrangeiro'})
pct_estrangeiro.head(6)

pct_estrangeiro
ano  classificacao_time_2 estrangeiro                 
2003 0                    1                  63.095238
                          0                  36.904762
     1                    1                  54.810496
                          0                  45.189504
     2                    0                  52.054795
                          1                  47.945205

In [34]:
pct_estrangeiro = pct_estrangeiro.reset_index()
pct_estrangeiro

,ano,classificacao_time_2,estrangeiro,pct_estrangeiro
0,2003,0,1,63.095238
1,2003,0,0,36.904762
2,2003,1,1,54.810496
3,2003,1,0,45.189504
4,2003,2,0,52.054795
...,...,...,...,...
109,2021,0,0,37.974684
110,2021,1,1,56.250000
111,2021,1,0,43.750000
112,2021,2,1,59.756098


In [35]:
# Filtrando apenas linhas de estrangeiros

pct_estrangeiro = pct_estrangeiro.query("estrangeiro == 1")
pct_estrangeiro.head()

,ano,classificacao_time_2,estrangeiro,pct_estrangeiro
0,2003,0,1,63.095238
2,2003,1,1,54.810496
5,2003,2,1,47.945205
6,2004,0,1,65.662651
8,2004,1,1,56.176471


## 2.9 Concatenando as bases em uma única e tratando a base

In [36]:
media_classificacao_time = pd.concat([media_classificacao_time, qtd_jogadores_posicao, pct_estrangeiro],axis=1,join='inner')
media_classificacao_time.head()

,ano,classificacao_time_2,idade,altura_jogador,peso_jogador,ano,classificacao_time_2,qtd_jogadores,time,media_jogadores,ano,classificacao_time_2,estrangeiro,pct_estrangeiro
0,2003,0,40.976190,1.818743,76.018072,2003,0,168,5,33.600000,2003,0,1,63.095238
2,2003,2,42.301370,1.812329,76.712329,2003,2,73,3,24.333333,2003,1,1,54.810496
5,2004,2,41.602410,1.820952,76.904762,2004,2,84,3,28.000000,2003,2,1,47.945205
6,2005,0,39.031250,1.827250,76.925000,2005,0,160,5,32.000000,2004,0,1,65.662651
8,2005,2,39.295775,1.824366,77.070423,2005,2,71,3,23.666667,2004,1,1,56.176471


In [37]:
media_classificacao_time = media_classificacao_time.loc[:,~media_classificacao_time.columns.duplicated()]
media_classificacao_time.head()

,ano,classificacao_time_2,idade,altura_jogador,peso_jogador,qtd_jogadores,time,media_jogadores,estrangeiro,pct_estrangeiro
0,2003,0,40.976190,1.818743,76.018072,168,5,33.600000,1,63.095238
2,2003,2,42.301370,1.812329,76.712329,73,3,24.333333,1,54.810496
5,2004,2,41.602410,1.820952,76.904762,84,3,28.000000,1,47.945205
6,2005,0,39.031250,1.827250,76.925000,160,5,32.000000,1,65.662651
8,2005,2,39.295775,1.824366,77.070423,71,3,23.666667,1,56.176471


In [38]:
# excluindo colunas desnecessárias

media_classificacao_time = media_classificacao_time.drop(columns = ['estrangeiro','time'])
media_classificacao_time.head()

,ano,classificacao_time_2,idade,altura_jogador,peso_jogador,qtd_jogadores,media_jogadores,pct_estrangeiro
0,2003,0,40.976190,1.818743,76.018072,168,33.600000,63.095238
2,2003,2,42.301370,1.812329,76.712329,73,24.333333,54.810496
5,2004,2,41.602410,1.820952,76.904762,84,28.000000,47.945205
6,2005,0,39.031250,1.827250,76.925000,160,32.000000,65.662651
8,2005,2,39.295775,1.824366,77.070423,71,23.666667,56.176471


In [39]:
media_classificacao_time.to_csv(url_loc_files + "media_classificacao_time_3.csv",index=False)